<a href="https://colab.research.google.com/github/SamuelTelesSilva/toxic_comment_detection/blob/main/notebooks/word_embedding_bert/BERT_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalação

In [ ]:
!pip install tensorflow
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 65.0 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 27.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns

import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
import scikitplot as skplt
from sklearn.metrics import classification_report

#BERT embeddings

##Using the BERT tokenizer / Usando o tokenizer BERT
- token das palavras
- id das palavras



Primeiro, importamos BertTokenizer. Modelos diferentes têm tokenizers diferentes; por exemplo, o tokenizer do modelo XLNet é chamado XLNetTokenizer.


Em segundo lugar, chamamos o método from_pretrained no objeto tokenizer e fornecemos o nome do modelo. Observe que não precisamos baixar manualmente o modelo **neuralmind/bert-base-portuguese-cased** pré-treinado; este método baixa o modelo sozinho.


Em seguida, chamamos o método tokenize. tokenize basicamente tokeniza a frase quebrando todas as palavras em subpalavras.


Imprimimos tokens para examinar as subpalavras. As palavras 'Eu', 'vou', 'comprar', 'um', 'carro', 'aman' '.' e assim por diante existem no vocabulário do Tokenizer e, portanto, são mantidas como estão. "amanhã" é uma palavra rara, então não existe no vocabulário do Tokenizer. Em seguida, o Tokenizer divide essa palavra nas subpalavras. Observe que "##hã" começa com os caracteres "##" para enfatizar o fato de que esta é uma palavra.


Em seguida, convertemos os tokens em seus IDs de token chamando convert_tokens_to_ids .

In [ ]:
sentence = "Eu vou comprar um carro amanhã."

In [ ]:
from transformers import BertTokenizer

btokenizer =\
BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased') #modelo em PT, modelo pré-treinado.

tokens = btokenizer.tokenize(sentence)
tokens

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

['Eu', 'vou', 'comprar', 'um', 'carro', 'aman', '##hã', '.']

In [ ]:
tokens

['Eu', 'vou', 'comprar', 'um', 'carro', 'aman', '##hã', '.']

In [ ]:
ids = btokenizer.convert_tokens_to_ids(tokens)
ids

[3396, 17891, 8977, 222, 3883, 22032, 5438, 119]

E quanto aos tokens [CLS] e [SEP], tokens especiais ao início e ao final da sentença de entrada. Para o código anterior, precisamos envolver mais uma etapa e adicionar nossos tokens especiais manualmente. Podemos fazer todas essas etapas de pré-processamento, o BERT fornece um método chamado **encode** que faz o seguinte

- Adiciona tokens **CLS** e **SEP** à frase de entrada.

 O primeiro token especial do BERT é [CLS]. O primeiro token de cada sequência de entrada deve ser [CLS]. Usamos esse token em tarefas de classificação como um agregado da sentença de entrada. Ignoramos esse token em tarefas sem classificação.

  [SEP] significa um separador de frases.

  [PAD] é um token especial que significa preenchimento

- Tokeniza a frase dividindo os tokens em subpalavras
- Converte os tokens em seus IDs de token

Esse segmento de código gera os IDs de token em apenas uma única etapa, em vez de chamar tokenize e convert_tokens_to_ids um após o outro. O resultado é uma lista Python.

In [ ]:
#Uma versão que pega o token e o id das palavras, o codigo anterior faz com etapas diferentes
from transformers import BertTokenizer

btokenizer =\
BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

sentence = "Eu vou comprar um carro amanhã."

#gera os IDs de token em apenas uma única etapa Token + id
#Já faz a tokenização e coloca os id das palavras
ids = btokenizer.encode(sentence)
ids


[101, 3396, 17891, 8977, 222, 3883, 22032, 5438, 119, 102]

todas as sentenças de entrada em um conjunto de dados devem ter o mesmo tamanho porque o BERT não pode processar sentenças de comprimento variável. Portanto, precisamos preencher as frases curtas com o comprimento da frase mais longa disponível no conjunto de dados. Além disso, se quisermos usar um tensor TensorFlow em vez de uma lista simples, precisamos escrever algum código de conversão. A biblioteca **HuggingFace** fornece **encode_plus** para facilitar nossa vida e combinar todas essas etapas em um método da seguinte forma:

In [ ]:
from transformers import BertTokenizer

btokenizer =\
BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

sentence = "Eu vou comprar um carro amanhã."

encoded = btokenizer.encode_plus(
  #frase de entrada
  text = sentence,
  #adiciona tokens especiais [CLS] e [SEP].
  add_special_tokens=True,
  #O tamanho máximo que você deseja que sua frase tenha. Se a sentença for menor que os tokens max_length, queremos preencher a sentença.
  max_length=12,
  #Alimentamos True se quisermos preencher a frase, caso contrário, False.
  pad_to_max_length=True,
  #Passamos este parâmetro se quisermos que a saída seja um tensor, caso contrário, a saída é uma lista Python. As opções disponíveis são tf e pt para TensorFlow e PyTorch
  return_tensors="tf"
)

token_ids = encoded["input_ids"]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(token_ids)

tf.Tensor([[  101  3396 17891  8977   222  3883 22032  5438   119   102     0     0]], shape=(1, 12), dtype=int32)


##Obtaining BERT word vectors / Obtendo vetores de palavras BERT

Este código é muito semelhante ao código anterior. Aqui, importamos TFbertModel . Depois disso, inicializamos o modelo BERT com o modelo pré-treinado, **neuralmind/bert-base-portuguese-cased**. Em seguida, transformamos nossa sentença de entrada para o formato de entrada BERT com encode_plus e capturamos o resultado, tf.tensor, na variável de inputs. Nós alimentamos nossa sentença para o modelo BERT e capturou essa saída com a variável de outputs.

In [ ]:
from transformers import BertTokenizer, TFBertModel
from transformers import AutoModelForPreTraining

btokenizer =\
BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

#Modelo do neuralmind é pytorch então tem que colocar o from_pt=True para utilizar o TensorFlow
bmodel = TFBertModel.from_pretrained("neuralmind/bert-base-portuguese-cased", from_pt=True)

sentence = "Eu vou comprar um carro amanhã."

encoded = btokenizer.encode_plus(
  text = sentence,
  add_special_tokens=True,
  max_length=12,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors="tf"
)

inputs = encoded["input_ids"]
outputs = bmodel(inputs)

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [ ]:
#O primeiro elemento da saída é a forma (tamanho do lote, comprimento da sequência, tamanho oculto).
#Alimentamos apenas uma frase, portanto, o tamanho do lote aqui é 1
#(1,max_length=12,768)
outputs[0].shape

TensorShape([1, 12, 768])

In [ ]:
outputs[1].shape

TensorShape([1, 768])

Excelente! Extraímos os embeddings do BERT. Em seguida, usaremos esses embeddings para treinar nosso modelo de classificação de texto com TensorFlow e tf.keras.